In [ ]:
import numpy as np
import math
import numpy.matlib as matlib
import matplotlib.pyplot as plt
import operator

from time import perf_counter
import scipy.stats

Note:

In the following, when the name of a function, an array, etc. contains the word 'tilde', noise is involved in some way.

# Function: SBPS MCMC

This is the function we use to run SBPS MCMC. It will output:
1. the turning points of the trajectory
2. velocities there at those points
3. times there at those points
4. total cpu seconds used there
5. total log-pdf evaluations used there

In [ ]:
# This is the function we call the SBPS MCMC.
# It will output the turning points of the trajectory, velocities, times there,
# cpu seconds used there, and number of pdf evaluations used there.



# num_lin_reg is the hyper-param for num of points used to do linear regression.
def SBPS(x0, v0, Time, lambda_ref, prob_dist, num_lin_reg, k, d_geom):
    
    cpu_time_executed = perf_counter()
    computational_times = [0]
    start_time = cpu_time_executed
    
    turn_pts = [x0]
    list_of_velo = [v0]
    striding_times = [0]
    total_evals_list = [1]
    dim = x0.size
    i = 1
    x = x0
    v = v0
    t = 0
    total_evals = 1
    
    next_singleton_ob_delta_U, G_tilde, c_t_squared, next_singleton_ob_log_grad_list =\
    eval_G_tilde_plus_emp_var(x, v, prob_dist)
    next_array_of_singleton_ob = np.array([[0, G_tilde, c_t_squared]])    # time; G_tilde; variance
    
    
    while t < Time:
        # Next bounce
        # next_array_of_obs is a np.array being [0, G_t, variance]
        tau_bounce, observed_grad, next_array_of_singleton_ob, next_singleton_ob_delta_U,\
        next_singleton_ob_log_grad_list, num_evals =\
        New_simulation_Cox_Process(x, v, next_array_of_singleton_ob, next_singleton_ob_delta_U, 
                                   next_singleton_ob_log_grad_list, prob_dist, num_lin_reg, k, d_geom)
        
        total_evals = total_evals + num_evals
        
        # Next refreshment
        beta = 1/lambda_ref
        tau_ref = np.random.exponential(scale = beta)
    
    
        tau = min(tau_bounce, tau_ref)
        x = x + tau*v
        t = t + tau
        
        
        if tau_ref < tau_bounce:
            v = np.random.standard_normal(dim)

        else:
            v = reflection(v, observed_grad)
            
        
        cpu_time_executed = perf_counter() - start_time
        
        list_of_velo.append(v)
        turn_pts.append(x)
        striding_times.append(t)
        computational_times.append(cpu_time_executed)
        total_evals_list.append(total_evals)

        if i%50 == 0:
          print('Complete {}th change of velocity'.format(i))
        i = i+1


    return turn_pts, list_of_velo, striding_times, total_evals_list, computational_times
    # x_list, v_list, t_list, evals_list, cpu_time_list
       

# Function: reflection of velocity

v is reflected according to a noisy gradient at some position.

In [ ]:

def reflection(v, observed_grad):
    
    return v - 2*(np.sum(observed_grad*v)/(np.sum(observed_grad*observed_grad)))*observed_grad

# Function: x_v_t_arbitrary_times

This can compute the position and velocity of a particle at any time points along the trajectory.

In [ ]:

def x_v_t_arbitrary_times(turn_pts, list_of_velo, striding_times, intermediate_times):
    
    num_changes = len(turn_pts)
    num_required_times = len(intermediate_times)
    tiling_interm_times = np.transpose(np.tile(intermediate_times, (num_changes, 1)))
    testing_mat_1 = tiling_interm_times - np.tile(striding_times, (num_required_times, 1))
    testing_mat_2 = np.where(testing_mat_1 >= 0, 1, 0)
    indices_no_later_than = np.sum(testing_mat_2, axis = 1) - 1
        
    turn_pts_no_later = [turn_pts[i] for i in indices_no_later_than]
    velo_no_later = [list_of_velo[i] for i in indices_no_later_than]
    stride_time_no_later = [striding_times[i] for i in indices_no_later_than]
    
       
    interm_times = list(intermediate_times)
        
    list_of_coasting_times = list(map(operator.sub, interm_times, stride_time_no_later))
    distance_strided = list(map(operator.mul, list_of_coasting_times, velo_no_later))
    locations_at_required_times = list(map(operator.add, turn_pts_no_later, distance_strided))
        

    return locations_at_required_times, velo_no_later, interm_times


# Function: Probability of Acceptance

A acceptance-rejection step is present in bounce-time simulation.

This function returns simply the probability of acceptance.

It aims to tackle the division-by-zero issue.




In [ ]:

def probability_of_acceptance(real_intensity, proposal_intensity):  # [G(t)]_{+}; lambda(t)
    
    if proposal_intensity > 0:
        return min(1, real_intensity/proposal_intensity)
    
    elif proposal_intensity == 0 and real_intensity > 0:
        return 1
    
    elif proposal_intensity == 0 and real_intensity == 0:
        return 0.5


# Function: eval_G_tilde_plus_emp_var

Given x, v, this function outputs the following:

1. the noisy energy gradient evaluated at x,
2. the noisy G value: (ener grad) dot product with (velocity)
3. the empirical variance of the G value
4. A list of gradient of log(something). See Equation (10).


In [ ]:
def eval_G_tilde_plus_emp_var(x, v, prob_dist):
    
    if prob_dist.nature_of_noise == 'artificial_gaus':
        
        delta_U_mean = prob_dist.ener_grad(x)
        gaus_noise_vec = np.random.multivariate_normal(np.zeros(x.size), np.identity(x.size)*prob_dist.exact_gaus_noise_var)
        
        delta_U_tilde = delta_U_mean + gaus_noise_vec
        # print('delta_U_tilde:', delta_U_tilde)
        # print('v: ', v)
        
        G_tilde = v.dot(delta_U_tilde)
        
        c_t_squared = (v.dot(v))*prob_dist.exact_gaus_noise_var
        
        log_grad_list = 'N/A'
        
        
    if prob_dist.nature_of_noise == 'subsampling':
        
        # log_grad is not the energy gradient; Equation (10) of the paper is followed.
        delta_U_tilde, log_grad_list = prob_dist.ener_grad(x, list_sampling = True)
        dot_prod_list = log_grad_list.dot(v)
        
        G_tilde = v.dot(delta_U_tilde)
        
        c_t_squared = (prob_dist.num_obs**2)*(1 - prob_dist.n/prob_dist.num_obs)*np.var(dot_prod_list)
        

    return delta_U_tilde, G_tilde, c_t_squared, log_grad_list



# Function: Bayesian Linear Regression - Approach 1 (assuming Gaussian Priors on beta0, beta1)

The following calculates the mean and covariance matrix of (beta_0, beta_1) of the posterior distribution,

after observations of G_tildes at different times are given.

In [ ]:

def Bayesian_Linear_Regression_doub_gaus(array_of_observations, sig_0, mu_0, sig_1, mu_1):
    
    # print(array_of_observations)
    
    array_of_times = array_of_observations[:,0]
    array_of_Gs = array_of_observations[:,1]
    array_of_variances = array_of_observations[:,2]
    
    S_1 = np.sum(1/array_of_variances)/2
    S_2 = np.sum((1/array_of_variances)*(array_of_times))/2
    S_3 = np.sum((1/array_of_variances)*(array_of_Gs))/2
    
    S_4 = np.sum((1/array_of_variances)*(array_of_times)*(array_of_Gs))/2
    S_5 = np.sum((1/array_of_variances)*(array_of_times)*(array_of_times))/2
    S_6 = np.sum((1/array_of_variances)*(array_of_Gs)*(array_of_Gs))/2
    
    
    D_00 = -2*S_1 - 1/(sig_0**2)
    D_01 = -2*S_2
    D_02 = mu_0/(sig_0**2) + 2*S_3
    
    D_10 = -2*S_2
    D_11 = -2*S_5 - 1/(sig_1**2)
    D_12 = mu_1/(sig_1**2) + 2*S_4
    
    
    if D_01 == 0:
        
        expect_beta0 = -D_02/D_00
        expect_beta1 = -D_12/D_11
        expect_beta0_squared = (D_02/D_00)**2 - 1/D_00
        expect_beta1_squared = (D_12/D_11)**2 - 1/D_11
        expect_beta0_beta1 = (D_02*D_12)/(D_00*D_11)
    
    
    
    else:
        expect_beta0 = (D_01*D_12 - D_02*D_11)/(D_00*D_11 - D_01*D_10)
        expect_beta1 = (D_10*D_02 - D_12*D_00)/(D_00*D_11 - D_01*D_10)
        expect_beta0_squared = ((D_01*D_12 - D_02*D_11)/(D_00*D_11 - D_01*D_10))**2 + D_11/(D_01*D_10 - D_00*D_11)
        expect_beta1_squared = ((D_10*D_02 - D_12*D_00)/(D_00*D_11 - D_01*D_10))**2 + D_00/(D_01*D_10 - D_00*D_11)
        expect_beta0_beta1 = expect_beta0*expect_beta1 + D_10/(D_00*D_11 - D_01*D_10)
    
    
    hat_beta_0 = expect_beta0
    hat_beta_1 = expect_beta1
    
    Var_0 = expect_beta0_squared - (expect_beta0)**2
    Var_1 = expect_beta1_squared - (expect_beta1)**2
    Cov_01 = expect_beta0_beta1 - (expect_beta0)*(expect_beta1)
    
    Sigma_cov_mat = np.array([[Var_0, Cov_01], [Cov_01, Var_1]])
    

    
    # need to calculate E[beta_0], E[beta_1], E[beta_1^2], E[beta_1*beta_2], E[beta_2^2]
    
    return hat_beta_0, hat_beta_1, Sigma_cov_mat



# Function: Bayesian Linear Regression - Approach 2 (center G_tildes without centering times)

In this approach, hat_beta_0 is taken to be the mean of G_tilde's. Only beta_1 (but not beta_0) is modelled as a random variable.

The major difference between this and the Bayesian Lasso paper's one is that this model does not center the times. I predict that the latter will perform better than this.

See pp. ??-?? of my report for more details.

In [ ]:
def Bayesian_Linear_Regression_Pakman(array_of_observations, sig_1, mu_1):
  # columns of array_of_observations
  array_of_times = array_of_observations[:,0]
  array_of_ys = array_of_observations[:,1]
  array_of_vars = array_of_observations[:,2]

  hat_beta_0 = np.mean(array_of_ys)


  P = 0.5/(sig_1**2) + 0.5*np.sum(array_of_times*array_of_times/array_of_vars)
  Q = np.sum(array_of_times*(hat_beta_0 - array_of_ys)/array_of_vars) - mu_1/(sig_1**2)


  hat_beta_1 = -0.5*Q/P
  post_cov_11 = 0.5/P

  Sigma_cov_mat = np.array([[0,0],[0,post_cov_11]])


  return hat_beta_0, hat_beta_1, Sigma_cov_mat

# Function: Bayesian Linear Regression - Approach 3 (following the Bayesian-Lasso paper)

In this approach, the time points [t_1, ..., t_m] are first standardised before the linear regression model is applied. See my report pp.??-?? for more details.

In [ ]:
def Bayesian_Linear_Regression_BayesLasso(array_of_observations, sig_1, mu_1):
  
  # standardising the time points
  array_of_times = array_of_observations[:,0]
  mean_of_times = np.mean(array_of_times)
  SD_of_times = np.sqrt(float(np.cov(array_of_times)))


  # mean of G_tilde's
  array_of_ys = array_of_observations[:,1]
  mean_of_ys = np.mean(array_of_ys)


  # array of empirical variance
  array_of_vars = array_of_observations[:,2]


  k_vec = (array_of_times - mean_of_times)/SD_of_times
  l_vec = mean_of_ys - array_of_ys


  P_tilde = 0.5/(sig_1**2) + 0.5*np.sum(k_vec*k_vec/array_of_vars)
  Q_tilde = np.sum(k_vec*l_vec/array_of_vars) - mu_1/(sig_1**2)


  hat_beta_1 = -0.5*Q_tilde/(SD_of_times*P_tilde)
  hat_beta_0 = mean_of_ys - hat_beta_1*mean_of_times
  post_cov_11 = 0.5/((SD_of_times**2)*P_tilde)

  Sigma_cov_mat = np.array([[0,0],[0,post_cov_11]])


  return hat_beta_0, hat_beta_1, Sigma_cov_mat

# Function: new_simulation_Cox_Process

In [ ]:

def New_simulation_Cox_Process(x, v, array_of_starting_singleton_ob, starting_singleton_delta_U, 
                               starting_singleton_log_grad_list, prob_dist, num_lin_reg, k, d_geom):

  dim = len(x)
  current_location = x
  
  termination = 0
  time_coasted = 0
  initialised_array_of_singleton_ob = array_of_starting_singleton_ob
  initialised_singleton_delta_U = starting_singleton_delta_U
  initialised_singleton_log_grad_list = starting_singleton_log_grad_list

  num_evals = 0
  t_geom = d_geom/np.linalg.norm(v)

  normalised_v = v/np.linalg.norm(v)
  tiling_v = np.tile(normalised_v, (num_lin_reg - 1, 1))
  mult_factors = np.transpose(np.tile(np.arange(1,num_lin_reg)*d_geom/(num_lin_reg-1), (dim,1)))
  moving_points_of_reg_obs = tiling_v*mult_factors

  moving_times_for_reg = np.arange(1,num_lin_reg)*d_geom/((num_lin_reg-1)*np.linalg.norm(v))

  # This counts the number of while loops performed in the following.
  regression_index = 0

  sig_0 = 1
  sig_1 = 1
  mu_0 = 0
  mu_1 = 0

  while termination == 0:
    # test whether the SBPS MCMC is stuck here
    # investigate why setting d_geom to low value is problematic.
    if regression_index % 100 == 99:
      print('Now, running the {}th segment of simulation of Cox Process'.format(regression_index))
      print('hat_beta_0: {}'.format(hat_beta_0))
      print('hat_beta_1: {}'.format(hat_beta_1))
      print('Sigma_cov_mat: {}'.format(Sigma_cov_mat))
    # Step 1: Bayesian Linear Regression

    points_of_regression_obs = current_location + moving_points_of_reg_obs
    
    array_for_lin_reg = initialised_array_of_singleton_ob      # 2D array of dimension (1,3)

    lin_reg_delta_U_list = [initialised_singleton_delta_U]

    if prob_dist.nature_of_noise == 'artificial_gaus':
      
      for i in range(num_lin_reg-1):
        BAYES_delta_U_tilde, BAYES_G_tilde, BAYES_c_t_squared, BAYES_log_grad_list =\
        eval_G_tilde_plus_emp_var(points_of_regression_obs[i], v, prob_dist)
        arr_observation = np.array([[moving_times_for_reg[i], BAYES_G_tilde, BAYES_c_t_squared]])
        
        array_for_lin_reg = np.concatenate((array_for_lin_reg, arr_observation), axis = 0)

        lin_reg_delta_U_list.append(BAYES_delta_U_tilde)

        # count the number of pdf evals
        num_evals = num_evals + 1

    if prob_dist.nature_of_noise == 'subsampling':
      
      sequence_of_log_grad_lists = [initialised_singleton_log_grad_list]

      for i in range(num_lin_reg-1):
        BAYES_delta_U_tilde, BAYES_G_tilde, BAYES_c_t_squared, BAYES_log_grad_list =\
        eval_G_tilde_plus_emp_var(points_of_regression_obs[i], v, prob_dist)
        arr_observation = np.array([[moving_times_for_reg[i], BAYES_G_tilde, BAYES_c_t_squared]])
        
        array_for_lin_reg = np.concatenate((array_for_lin_reg, arr_observation), axis = 0)
        
        lin_reg_delta_U_list.append(BAYES_delta_U_tilde)

        sequence_of_log_grad_lists.append(BAYES_log_grad_list)

        # count the number of pdf evals
        num_evals = num_evals + 1
    
    hat_beta_0, hat_beta_1, Sigma_cov_mat =\
    Bayesian_Linear_Regression_doub_gaus(array_for_lin_reg, sig_0, mu_0, sig_1, mu_1)
    


    # Step 2: Propose events from this upper NHPP. Perform acceptance-rejection
    # The first arrival of an NHPP with time duration [0, d_geom/|v|] is simulated.
    
    if prob_dist.nature_of_noise == 'artificial_gaus':
      event_time, delta_U_tilde, G_tilde, c_t_squared, log_grad_list, num_evals_ADAPTIVE =\
      analytic_Bi_adaptive_thinning(current_location, v, array_for_lin_reg, lin_reg_delta_U_list, 
                                    hat_beta_0, hat_beta_1, Sigma_cov_mat, prob_dist, 
                                    num_lin_reg, k, d_geom)
      num_evals = num_evals + num_evals_ADAPTIVE

    if prob_dist.nature_of_noise == 'subsampling':
      event_time, delta_U_tilde, G_tilde, c_t_squared, log_grad_list, num_evals_ADAPTIVE =\
      analytic_Bi_adaptive_thinning(current_location, v, array_for_lin_reg, lin_reg_delta_U_list, 
                                    hat_beta_0, hat_beta_1, Sigma_cov_mat, prob_dist, 
                                    num_lin_reg, k, d_geom, nature_of_noise='subsampling', 
                                    sequence_of_log_grad_lists = sequence_of_log_grad_lists)
      num_evals = num_evals + num_evals_ADAPTIVE

    
    # Step 3: if a bounce occur, hurray! If not, there are things to do...
    if (not event_time == 'no_events'):
      time_coasted = time_coasted + event_time

      if prob_dist.nature_of_noise == 'artificial_gaus':
        next_array_of_singleton_ob = np.array([[0, -G_tilde, c_t_squared]])
        next_singleton_ob_delta_U = delta_U_tilde
        next_singleton_ob_log_grad_list = 'N/A'

      
      if prob_dist.nature_of_noise == 'subsampling':
        reflected_v = reflection(v, delta_U_tilde)
        Equation_10_last_bit = log_grad_list.dot(reflected_v)
        new_c_t_squared = (prob_dist.num_obs**2)*(1 - prob_dist.n/prob_dist.num_obs)*np.var(Equation_10_last_bit)

        next_array_of_singleton_ob = np.array([[0, -G_tilde, new_c_t_squared]])
        next_singleton_ob_delta_U = delta_U_tilde
        next_singleton_ob_log_grad_list = log_grad_list

      termination = 1
    
    elif event_time == 'no_events':
      # update prior beliefs about mu_0, mu_1 (idea behind is because of the continuously differentiable
      # geometry of the energy function).
#      mu_0 = hat_beta_0
#      mu_1 = hat_beta_1

      current_location = current_location + d_geom*normalised_v
      INITIAL_delta_U_tilde, INITIAL_G_tilde, INITIAL_c_t_squared, INITIAL_log_grad_list =\
      eval_G_tilde_plus_emp_var(current_location, v, prob_dist)
      initialised_array_of_singleton_ob = np.array([[0, INITIAL_G_tilde, INITIAL_c_t_squared]])

      initialised_singleton_delta_U = INITIAL_delta_U_tilde
      initialised_singleton_log_grad_list = INITIAL_log_grad_list


      num_evals = num_evals + 1

      time_coasted = time_coasted + t_geom

    regression_index = regression_index + 1



  return time_coasted, delta_U_tilde, next_array_of_singleton_ob, next_singleton_ob_delta_U, next_singleton_ob_log_grad_list, num_evals







# Function: analytic_Bi_adaptive_thinning

analytic_Bi stands for "analytic bisection".

In [ ]:
# Let Lambda be the upper-bound intensity function.
# Let Lambda = max(0, Rho).

def analytic_Bi_adaptive_thinning(x, v, array_for_lin_reg, lin_reg_delta_U_list, hat_beta_0, hat_beta_1, 
                                  Sigma_cov_mat, prob_dist, num_lin_reg, k, d_geom, 
                                  nature_of_noise='artificial_gaus', 
                                  sequence_of_log_grad_lists = 'None', num_bisections = 17):
  
  num_evals_ADAPTIVE = 0

  # Step 0: define the Rho function
  Rho = lambda t: hat_beta_0 + hat_beta_1*t + k*np.sqrt(Sigma_cov_mat[0,0] + 2*Sigma_cov_mat[0,1]*t 
                                                        + Sigma_cov_mat[1,1]*(t**2))



  # Step 1: find all global time t such that Rho(t)=0. There is an exception
  #         when A=0, B=0, C=0.
  A, B, C, candidates = quadratic_equation_candidates_only(hat_beta_0, hat_beta_1, Sigma_cov_mat, k)
  
  






  # Step 2: Lambda = max(0, Rho). We need to determine the sign of Rho in different regions of [0,t_geom],
  #         by picking a point (the variable test_point below) and evaluate Rho on it.
  t_geom = d_geom/np.linalg.norm(v)

  if candidates == 'no_candidates':
    boundaries_and_zeroes = [0, t_geom]
  else:
    # a new condition of (hat_beta_0 + hat_beta_1*x < 0) is added to reject roots.
    filter_candidates = [x for x in candidates if (0 < x) and (x < t_geom) 
                         and (hat_beta_0 + hat_beta_1*x < 0)]
    boundaries_and_zeroes = [0] + filter_candidates + [t_geom]
  
  B_and_Z_length = len(boundaries_and_zeroes)
  
  # 1 means that the region has a positive sign of Rho. 0 means otherwise.
  signs_of_regions = []

  for i in range(B_and_Z_length - 1):
    test_point = 0.5*(boundaries_and_zeroes[i] + boundaries_and_zeroes[i+1])

    if Rho(test_point) > 0:
      signs_of_regions.append(1)
    else:
      signs_of_regions.append(0)



  # Step 3: Note that Rho can be integrated analytically. Hence, We need to construct an 
  # anti-derivative for it.
  a = Sigma_cov_mat[1,1]
  b = Sigma_cov_mat[0,0] - (Sigma_cov_mat[0,1]**2)/Sigma_cov_mat[1,1]
  c = Sigma_cov_mat[0,1]/Sigma_cov_mat[1,1]

  complicated_1 = lambda t: (t + c)*np.sqrt(a*((t + c)**2) + b)
  complicated_2 = lambda t: (b/np.sqrt(a))*np.log(np.sqrt(a)*np.sqrt(a*((t + c)**2) + b) + a*(t + c))   if (not b == 0)   else  0

  Anti_derivative = lambda t: hat_beta_0*t + 0.5*hat_beta_1*(t**2) +\
  0.5*k*(complicated_1(t) + complicated_2(t))



  # Step 4: compute intensity in each region.
  intensities_of_individual_regions = []
  cumulative_intensities_of_each_region = [0]
  
  for i in range(B_and_Z_length - 1):
    if signs_of_regions[i] == 0:
      intensities_of_individual_regions.append(0)
      
      cumulative_intensities_of_each_region.append(cumulative_intensities_of_each_region[-1])

    if signs_of_regions[i] == 1:
      regional_intensity = (Anti_derivative(boundaries_and_zeroes[i+1]) - 
                            Anti_derivative(boundaries_and_zeroes[i]))
      
      intensities_of_individual_regions.append(regional_intensity)
      cumulative_intensities_of_each_region.append(cumulative_intensities_of_each_region[-1] + 
                                                   regional_intensity)

  cumulative_intensities_of_each_region = cumulative_intensities_of_each_region[1:]




  # Step 5: Simulate the first arrival of the Cox Process via adaptive thinning on [0, t_geom].
  #         Note that it is possible that our upper-bound NHPP (or our Cox Process) has no 
  #         arrivals at all.


  # We are going to simulate events of our upper-bound NHPP in [0, t_geom] until having an acceptance,
  # or until that the whole period [0, t_geom] has been simulated. We will accumulate the 'rejected' 
  # inverse-transform intensities in this variable.
  accumulated_intensity = 0

  termination = 0

  while termination == 0:
    V = np.random.uniform()*(-1) + 1
    accumulated_intensity = accumulated_intensity - np.log(V)



    # this is an indicator for absence of events in NHPP
    i_no_occurrence = 1
    ith_region = 0

    while (i_no_occurrence == 1) and (ith_region <= (B_and_Z_length - 2)):
      if accumulated_intensity == cumulative_intensities_of_each_region[ith_region]:
        proposal_event_time = boundaries_and_zeroes[ith_region+1]
        i_no_occurrence = 0
    
      elif accumulated_intensity < cumulative_intensities_of_each_region[ith_region]:
        if ith_region == 0:
          remaining_intensity = accumulated_intensity
        
        elif ith_region > 0:
          remaining_intensity = accumulated_intensity - cumulative_intensities_of_each_region[ith_region-1]
      
        # We consider the following function to have the following interval domain:
        # [current_region_lower_bound, current_region_upper_bound]
        twisted_integral = lambda t: Anti_derivative(t) - Anti_derivative(boundaries_and_zeroes[ith_region])\
        - remaining_intensity
      
        proposal_event_time = Bisection_Method_strict_increase(twisted_integral, boundaries_and_zeroes[ith_region], 
                                                             boundaries_and_zeroes[ith_region+1], num_bisections)
        i_no_occurrence = 0
      
      ith_region = ith_region + 1
    
    if i_no_occurrence == 1:
      event_time = 'no_events'
      delta_U_tilde = 'N/A'
      G_tilde = 'N/A'
      c_t_squared = 'N/A'
      log_grad_list = 'N/A'
      termination = 1
  
    elif i_no_occurrence == 0:
      # acceptance/rejection step
      proposal_intensity = max(0, Rho(proposal_event_time))
      proposal_location = x + v*proposal_event_time


      # A situation that is unlikely to occur in the real computational environment: proposal_event_time
      # is one of those times used for performing linear regression [0, t_geom/(k-1), ..., t_geom].
      # If this really occurs, then we should not re-evaluate noisy gradient and G_tilde again.
      # Inside each nature_of_noise case, I tackle this unlikely situation, because I feel responsible
      # for coding in more things for the research of the PINTS group.
      
      if nature_of_noise == 'artificial_gaus':

        # i_lin_reg_time is an indicator for whether the proposal time is a time instant
        # used in performing linear regression.
        i_lin_reg_time = 0
        lin_reg_time_list = array_for_lin_reg[:,0]
        
        # debugging
#        if (not np.array_equal(lin_reg_time_list, np.arange(num_lin_reg)*t_geom/(num_lin_reg-1))):
#          print('error in analytic_Bi: timing of linear regression')
#          print('lin_reg_time_list is: {}'.format(lin_reg_time_list))
#          print('The supposed-to-be-correct is: {}'.format(np.arange(num_lin_reg)*t_geom/(num_lin_reg-1)))
#        if (not len(lin_reg_time_list) == num_lin_reg):
#          print('error in analytic_Bi: length of timing of linear regression')
      
        for i in range(num_lin_reg):
          if proposal_event_time == lin_reg_time_list[i]:
            i_lin_reg_time = ['revisited ith', i]

        if (not i_lin_reg_time == 0):
          lin_reg_time, G_tilde, c_t_squared = array_for_lin_reg[i_lin_reg_time[1]]
          delta_U_tilde = lin_reg_delta_U_list[i_lin_reg_time[1]]

          log_grad_list = 'N/A'
        

        elif i_lin_reg_time == 0:
          delta_U_tilde, G_tilde, c_t_squared, log_grad_list =\
          eval_G_tilde_plus_emp_var(proposal_location, v, prob_dist)

          num_evals_ADAPTIVE = num_evals_ADAPTIVE + 1

        real_intensity = max(0, G_tilde)
        
        U = np.random.uniform()
        if U <= probability_of_acceptance(real_intensity, proposal_intensity):
          event_time = proposal_event_time
          termination = 1

      


      if nature_of_noise == 'subsampling':

        i_lin_reg_time = 0
        lin_reg_time_list = array_for_lin_reg[:,0]
        
        # debugging
#        if (not lin_reg_time_list == np.arange(num_lin_reg)*t_geom/(num_lin_reg-1)):
#          print('error in analytic_Bi: timing of linear regression')
#          print('lin_reg_time_list is: {}'.format(lin_reg_time_list))
#        if (not len(lin_reg_time_list) == num_lin_reg):
#          print('error in analytic_Bi: length of timing of linear regression')
      
        for i in range(num_lin_reg):
          if proposal_event_time == lin_reg_time_list[i]:
            i_lin_reg_time = ['revisited ith', i]

        if (not i_lin_reg_time == 0):
          lin_reg_time, G_tilde, c_t_squared = array_for_lin_reg[i_lin_reg_time[1]]
          delta_U_tilde = lin_reg_delta_U_list[i_lin_reg_time[1]]

          log_grad_list = sequence_of_log_grad_lists[i_lin_reg_time[1]]
        

        elif i_lin_reg_time == 0:
          delta_U_tilde, G_tilde, c_t_squared, log_grad_list =\
          eval_G_tilde_plus_emp_var(proposal_location, v, prob_dist)

          num_evals_ADAPTIVE = num_evals_ADAPTIVE + 1

        real_intensity = max(0, G_tilde)
        
        U = np.random.uniform()
        if U <= probability_of_acceptance(real_intensity, proposal_intensity):
          event_time = proposal_event_time
          termination = 1
  


  return event_time, delta_U_tilde, G_tilde, c_t_squared, log_grad_list, num_evals_ADAPTIVE


# Function: Roots of Quadratic Equations

This function finds possible candidate of real roots of certain quadratic equations with some features, Ax^2+Bx+C=0.

It also contain the step of rejecting candidate time t such that Rho(t) is nonzero.

For more info, please refer to the MSc report (pp. ??-??).

In [ ]:
def quadratic_equation_candidates_only(hat_beta_0, hat_beta_1, Sigma_cov_mat, k):

  A = (k**2)*Sigma_cov_mat[1,1] - hat_beta_1**2
  B = 2*((k**2)*Sigma_cov_mat[0,1] - hat_beta_0*hat_beta_1)
  C = (k**2)*Sigma_cov_mat[0,0] - hat_beta_0**2

  if abs(A) < 10**(-10):
    A = 0
  if abs(B) < 10**(-10):
    B = 0
  if abs(C) < 10**(-10):
    C = 0


  if A==0 and B==0 and C==0:
#    print('1')
    candidates = [-hat_beta_0/hat_beta_1]


  elif A==0 and B==0 and (not C==0):
#    print('2')
    candidates = 'no_candidates'


  elif A==0 and (not B==0) and (not C==0):
#    print('3')
    # check if (-C/B) really produces zero intensity
    if hat_beta_0 + hat_beta_1*(-C/B) < 0:
      candidates = [-C/B]
    else:
      candidates = 'no_candidates'
  

  elif A==0 and (not B==0) and C==0:
#    print('4')
    candidates = 'no_candidates'



  else:      # this means that A is non-zero:
#    print('5')
    discriminant = B**2 - 4*A*C
    
    if discriminant < 0:
#      print('5.1')
      candidates = 'no_candidates'
    
    elif discriminant == 0:
#      print('5.2')
      sol = -B/(2*A)
      if hat_beta_0 + hat_beta_1*(sol) < 0:
        candidates = [sol]
      else:
        candidates = 'no_candidates'



    
    elif discriminant > 0:
#      print('5.3')
      sol_1 = -B/(2*A) - np.sqrt(discriminant)/(2*A)
      sol_2 = -B/(2*A) + np.sqrt(discriminant)/(2*A)

      if hat_beta_0 + hat_beta_1*(sol_1) < 0 and hat_beta_0 + hat_beta_1*(sol_2) < 0:
        candidates = [sol_1, sol_2]

      elif hat_beta_0 + hat_beta_1*(sol_1) < 0 and (not hat_beta_0 + hat_beta_1*(sol_2) < 0):
        candidates = [sol_1]

      elif (not hat_beta_0 + hat_beta_1*(sol_1) < 0) and hat_beta_0 + hat_beta_1*(sol_2) < 0:
        candidates = [sol_2]

      else:
        candidates = 'no_candidates'


  return A, B, C, candidates


# Function: Bisection Method

Because it seems that there are some unwanted complications involving xtol, rtol and maxiter of scipy.optimize.bisect (see https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.bisect.html),

I will build my own bisection method.

We want to find a root of F within [a,b].

We will only work with strictly increasing F.

We also assume that (F(a) <= 0) and (F(b) >= 0).

In [ ]:
# num_iters is the number of bisections to be performed.


def Bisection_Method_strict_increase(F, a, b, num_iters):
  exact_sol = 'bla'
  i_exact_sol = 0
  
  if F(a) == 0:
    exact_sol = a
    i_exact_sol = 1
  
  elif F(b) == 0:
    exact_sol = b
    i_exact_sol = 1
  
  else:
    lower_bound = a
    upper_bound = b

    for i in range(num_iters):
      mid_point = 0.5*(lower_bound + upper_bound)

      if F(mid_point) == 0:
        exact_sol = mid_point
        i_exact_sol = 1
        break

      elif F(mid_point) < 0:
        lower_bound = mid_point
      
      elif F(mid_point) > 0:
        upper_bound = mid_point
    
    approx_sol = 0.5*(lower_bound + upper_bound)

  
  if i_exact_sol == 1:
    return exact_sol

  elif i_exact_sol == 0:
    return approx_sol



# Noisy Gaussian distribution

In [ ]:
class Noisy_Gaussian:
    
    def __init__(self, mean, cov_mat, noise_presence, exact_gaus_noise_var, 
                 nature_of_noise = 'artificial_gaus'):
        self.mean = mean
        self.cov_mat = cov_mat
        self.dim = mean.size

        self.noise_presence = noise_presence
        self.exact_gaus_noise_var = exact_gaus_noise_var
        self.nature_of_noise = 'artificial_gaus'
    
    
    def energy(self, x): # constant coeff is discarded
        return 0.5*((x - self.mean).dot(np.linalg.inv(self.cov_mat)).dot(x - self.mean)) 
        
   
    def ener_grad(self, x):
#      if noise_presence == True:
#        noise = np.sqrt(self.exact_gaus_noise_var)*np.random.standard_normal(self.dim)
#        return (np.linalg.inv(self.cov_mat)).dot(x - self.mean) + noise
#      
#      else:
      return (np.linalg.inv(self.cov_mat)).dot(x - self.mean)